<a href="https://www.kaggle.com/code/niramay/obama-truth-lie-classification?scriptVersionId=112479297" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np 
import pandas as pd 
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/obama-quotestruths-lies/politifact-obama.csv


In [2]:
data = pd.read_csv('/kaggle/input/obama-quotestruths-lies/politifact-obama.csv')
data.head()

,Score,Date,Quote
0,False,"July 22, 2021","""The cost of an automobile, it's kind of back ..."
1,False,"June 23, 2021","""The Second Amendment, from the day it was pas..."
2,False,"May 3, 2021",For vaccine rates among Americans 65 and older...
3,False,"March 25, 2021",“We’re sending back the vast majority of the f...
4,False,"February 16, 2021","""If we kept (the minimum wage) indexed to infl..."


In [3]:
data = data.drop("Date", axis = 1)
data.head()

,Score,Quote
0,False,"""The cost of an automobile, it's kind of back ..."
1,False,"""The Second Amendment, from the day it was pas..."
2,False,For vaccine rates among Americans 65 and older...
3,False,“We’re sending back the vast majority of the f...
4,False,"""If we kept (the minimum wage) indexed to infl..."


In [4]:
data['Quote'] = data['Quote'].str.lower()

In [5]:
stop = stopwords.words('english')
data['Quote'] = data['Quote'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data.head()

,Score,Quote
0,False,"""the cost automobile, kind back pandemic."""
1,False,"""the second amendment, day passed, limited typ..."
2,False,"vaccine rates among americans 65 older, “there..."
3,False,“we’re sending back vast majority families com...
4,False,"""if kept (the minimum wage) indexed inflation,..."


In [6]:
data['Quote']= data['Quote'].apply(lambda x : re.sub('<.*?>','',x))
data.head()

,Score,Quote
0,False,"""the cost automobile, kind back pandemic."""
1,False,"""the second amendment, day passed, limited typ..."
2,False,"vaccine rates among americans 65 older, “there..."
3,False,“we’re sending back vast majority families com...
4,False,"""if kept (the minimum wage) indexed inflation,..."


In [7]:
data['Score'].value_counts()

Half True        211
Mostly True      211
True             148
False            107
Mostly False     101
Pants on Fire     15
Score              1
Name: Score, dtype: int64

In [8]:
data['check'] = data['Score'].replace({"Mostly True": 1, "True":1, "Half True":0, "Mostly False":0,"False":0,"Pants on Fire":0,"Score":0})
data['check'].value_counts()

0    435
1    359
Name: check, dtype: int64

In [9]:
data.head()

,Score,Quote,check
0,False,"""the cost automobile, kind back pandemic.""",0
1,False,"""the second amendment, day passed, limited typ...",0
2,False,"vaccine rates among americans 65 older, “there...",0
3,False,“we’re sending back vast majority families com...,0
4,False,"""if kept (the minimum wage) indexed inflation,...",0


In [10]:
data.drop('Score',axis = 1, inplace = True)
data.head()

,Quote,check
0,"""the cost automobile, kind back pandemic.""",0
1,"""the second amendment, day passed, limited typ...",0
2,"vaccine rates among americans 65 older, “there...",0
3,“we’re sending back vast majority families com...,0
4,"""if kept (the minimum wage) indexed inflation,...",0


In [11]:
all_quotes = list(data["Quote"])
len(all_quotes)

794

In [12]:
checks = list(data['check'])
len(checks)

794

In [13]:
X = all_quotes
y = checks
X_train, X_test,Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state = 45)

In [14]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

words_to_index = tokenizer.word_index

In [15]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2022-11-29 19:31:10--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-11-29 19:31:10--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-29 19:31:10--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [16]:
def read_glove_vector(glove_vec):
    with open(glove_vec, 'r', encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            w_line = line.split()
            curr_word = w_line[0]
            word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



    return word_to_vec_map

In [17]:
word_to_vec_map = read_glove_vector('/kaggle/working/glove.6B.50d.txt')

maxLen = 150

In [18]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        emb_matrix[index, :] = embedding_vector
embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

In [19]:

def def_model(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model